In [4]:
#Python code to retrieve HTML content from URL and extract text - Generic Python code no LLM Prompt here

import requests
import pdfplumber
import io

from bs4 import BeautifulSoup

# Step 1a: Define function to fetch and clean text HTML URL
def fetch_and_clean_html_text(url):
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status() #Raises HTTP Error for Bad Responses
    except requests.RequestException as e:
        print(f"Failed to fetch URL {url}: {e}")
        return None

    soup = BeautifulSoup(response.text, 'html.parser')
    raw_text = soup.get_text()
    cleaned_text = '\n'.join(
        line.strip() for line in raw_text.splitlines() if line.strip()
    )
    return cleaned_text


# Step 1b: Define function to fetch and clean text from PDF URL
def fetch_pdf_text(url):
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status() #Raises HTTP Error for Bad Responses
    except requests.RequestException as e:
        print(f"Failed to fetch URL {url}: {e}")
        return None

        with pdfplumber.open(io.BytesIO(response.content)) as pdf:
            all_text = ""
            for page in pdf.pages:
                text = page.extract_text()
                if text:
                    all_text += text + "\n"
        return all_text.strip()

# The following changes deal with Structured Tables within the PDF
def fetch_pdf_table(url):
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status() #Raises HTTP Error for Bad Responses
    except requests.RequestException as e:
        print(f"Failed to fetch URL {url}: {e}")
        return None

    output_text = ""
    with pdfplumber.open(io.BytesIO(response.content)) as pdf:
        for i, page in enumerate(pdf.pages):
            tables = page.extract_tables()
            for table in tables:
                for row in table:
                    if row:
                        line = " | ".join(cell.strip() if cell else "" for cell in row)
                        output_text += line + "\n"
    return output_text.strip()


# Step 2; Fetch the specs from the URLs
#url1 = "https://www.qualcomm.com/products/mobile/snapdragon/smartphones/snapdragon-8-series-mobile-platforms/snapdragon-8s-gen-4-mobile-platform#features"
#url2 = "https://www.qualcomm.com/products/mobile/snapdragon/smartphones/snapdragon-6-series-mobile-platforms/snapdragon-6-gen-4-mobile-platform"

url1 = "https://download.intel.com/newsroom/2025/c8e6h3a2/intel-q4-2024y-earnings.pdf"
url2 = "https://d1io3yog0oux5.cloudfront.net/_072085c6aa5f5433136803eee94e2b25/amd/db/841/9090/financial_tables_pdf/Q4+2024+Gaap+and+Non-Gaap+Earnings+Tables.pdf"

# Create alistof URLs and Iterate - for later Step. dummy placeholder code for now
finance_report_urls = [url1, url2, ...]


text_content_1 = fetch_pdf_text(url1) or ""
table_content_1 = fetch_pdf_table(url1) or ""
combined_content_1 = text_content_1 + "\n\n" + table_content_1

text_content_2 = fetch_pdf_text(url2) or ""
table_content_2 = fetch_pdf_table(url2) or ""
combined_content_2 = text_content_2 + "\n\n" + table_content_2

print(combined_content_1)
print(combined_content_2)


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def



Non-GAAP
adjustment or
measure | Definition | Usefulness to management and investors
Acquisition-related
adjustments | Amortization of acquisition-related intangible assets
consists of amortization of intangible assets such
as developed technology, brands, and customer
relationships acquired in connection with business
combinations. Charges related to the amortization
of these intangibles are recorded within both cost of
sales and MG&A in our US GAAP financial
statements. Amortization charges are recorded
over the estimated useful life of the related
acquired intangible asset, and thus are generally
recorded over multiple years. | We exclude amortization charges for our acquisition-
related intangible assets for purposes of calculating
certain non-GAAP measures because these charges
are inconsistent in size and are significantly impacted
by the timing and valuation of our acquisitions. These
adjustments facilitate a useful evaluation of our
current operating performance and compariso

In [6]:
# Initialize Setup Cell. Getting the OpenAPI Keys...Do I need something similar for other Models & make this a generic wrapper ?

import os
from dotenv import load_dotenv, find_dotenv

_= load_dotenv(find_dotenv()) # read local .env file

# Global settings
PROVIDER = os.getenv("PROVIDER", "openrouter").lower()
MODEL = os.getenv("MODEL", "mistralai/mixtral-8x7b-instruct")

def get_completion(prompt, model=None, temperature=0):
    model = model or MODEL
    messages = [{"role": "user", "content": prompt}]

    if PROVIDER == "openai":
        from openai import OpenAI
        client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            temperature=temperature,
        )
        return response.choices[0].message.content.strip()

    elif PROVIDER == "openrouter":
        from openai import OpenAI  # still uses OpenAI SDK
        client = OpenAI(
            api_key=os.getenv("OPENROUTER_API_KEY"),
            base_url="https://openrouter.ai/api/v1",
        )
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            temperature=temperature,
        )
        return response.choices[0].message.content.strip()

    elif PROVIDER == "anthropic":
        import anthropic
        client = anthropic.Anthropic(api_key=os.getenv("ANTHROPIC_API_KEY"))
        response = client.messages.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=1024,
            temperature=temperature,
        )
        return response.content[0].text.strip()

    elif PROVIDER == "mistral":
        from mistralai.client import MistralClient
        client = MistralClient(api_key=os.getenv("MISTRAL_API_KEY"))
        response = client.chat(
            model=model,
            messages=messages,
            temperature=temperature,
        )
        return response.choices[0].message.content.strip()

    elif PROVIDER == "groq":
        import groq
        client = groq.Groq(api_key=os.getenv("GROQ_API_KEY"))
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            temperature=temperature,
        )
        return response.choices[0].message.content.strip()

    else:
        raise ValueError(f"Unsupported provider: {PROVIDER}")



prompt = f"""
Your task is to parse the financial report of the company provided and do the following

1. Extract Key financial KPIS:

  Overall:
  
 - Revenue
 - Operating Expenses
 - Operating  Margin
 - EBITDA
 - Net Income
 - Operating Margin

 Profitability:
 
 -  Gross Profit Margin:
 -  Net Profit Margin: (Net Income / Revenue) 
 -  Return on Equity (ROE): (Net Income / Shareholders' Equity) 
 -  Return on Assets (ROA): (Net Income / Total Assets)
    
 - YoY Growth %
 - EPS (Earnings per Share)
 - FCF (Free Cash Flow)
 - Debt Levels
 - 
 
Highlight other KPIs as required.
If there is anything unusual that might highlight growth, risks and other factors highlight those.
The Financial Reports of the company are delimited by triple backticks

Company Financial Report: ```{combined_content_1}```

"""
response = get_completion(prompt)
print(response)



Based on the provided non-GAAP financial report, here are the key financial KPIs:

Overall:

* Revenue: Not provided in the report.
* Operating Expenses: Not explicitly provided, but can be calculated as the sum of Cost of Sales, Research and Development, and Marketing and Administrative expenses, excluding acquisition-related intangible assets amortization and restructuring charges.
* Operating Margin: Not explicitly provided, but can be calculated as Operating Income / Revenue, excluding acquisition-related intangible assets amortization and restructuring charges.
* EBITDA: Not explicitly provided, but can be calculated as Operating Income + Depreciation and Amortization, excluding acquisition-related intangible assets amortization and restructuring charges.
* Net Income: Not explicitly provided, but can be calculated as Net Income, excluding acquisition-related intangible assets amortization, restructuring charges, (gains) losses on equity investments, (gains) losses from divestitur